# Renewable Ninja API Client
Renewable Ninja provides API access to renewable energy data. This notebook demonstrates how to use the API to retrieve data from the Renewable Ninja database. The API is documented at https://api.renewables.ninja. Before running the notebook, you need to create an account at https://www.renewables.ninja. The API key can be found in the user profile page. It is a string of 32 characters and is used to authenticate the user. The API key is passed as a parameter ``token`` in the notebook.

The location of the data is specified by the latitude and longitude of the location. The latitude and longitude are passed as parameters ``lat`` and ``lon`` in the notebook. The latitude and longitude are specified in decimal degrees. Those data is written  in **locations.csv** file. The coordinates are taken from Google Maps.

## Prerequisites
This notebook requires the following packages:
- ``requests``, for making request to the API
- ``csv``, for reading and writing csv files
- ``os``, for creating directories
- ``ratelimit``, for limiting the number of requests per second

I recommend using the virtual environment to install the packages and run the notebook. The virtual environment can be created according to this [article](https://janakiev.com/blog/jupyter-virtual-envs/).

In [142]:
# Class Place to store the name, longitude and latitude of the places

class Place:
    def __init__(self, name: str, longitude: str, latitude: str):
        self.name = name
        self.longitude = longitude
        self.latitude = latitude

    def get_longitudes(places: list) -> list:
        longitudes = []
        for place in places:
            longitudes.append(place.longitude)
        return longitudes

    def get_latitudes(places: list) -> list:
        latitudes = []
        for place in places:
            latitudes.append(place.latitude)
        return latitudes

locations = []
locations_input = "locations.csv"
print("The locations from the csv file, which contains the name, longitude and latitude of the places.")
with open(locations_input, "r") as csv_file:
    for line in csv_file:
        line = line.strip()
        line = line.split(",")
        locations.append(Place(line[0], line[1], line[2]))
        print(line)
csv_file.close()

The locations from the csv file, which contains the name, longitude and latitude of the places.
['IE0', '52.71393469531283', ' -7.548505556345182']
['GB5', '54.61005097831343', ' -5.994804087604594']
['GB2', '55.95491046630958', ' -3.6021038257441016']
['GB4', '51.62769925163615', ' -3.011697267622681']
['GB1', '53.10746017233778', ' -2.203772503877576']
['GB3', '54.06657801985359', ' -0.5568489470125563']
['GB0', '51.453758824031084', ' 0.15785372860810987']
['ES0', '42.744807308016355', ' -5.4567506930404095']
['ES2', '37.945924870242806', ' -5.648658099341088']
['ES1', '40.86146554373322', ' -0.05135874890460168']
['ES4', '39.64096060941395', ' 2.6993140747384707']
['FR6', ' 43.746597713712866', ' 0.04459495424574143']
['FR3', '43.90151536002772', ' 5.2350389544952876']
['FR4', '47.441984148212', ' -1.5326410584313435']
['FR1', '45.315206581380544', ' 3.013985668399676']
['FR5', '45.69976373942615', ' 5.9405499983138945']
['FR2', '49.537031656642576', ' 2.5959050498405025']
['BE0', 

In [143]:
#prepare libraries
import requests
import csv
import os

# Create the folder according to the name of the places
for location in locations:
    if not os.path.exists(location.name[:2]):
        os.makedirs(location.name[:2],exist_ok=True)

In [144]:
#prepare the api token and url
#change the token to your own token
token = ''
api_base = 'https://www.renewables.ninja/api/'

In [145]:
#prepare the session
s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}

url = api_base + 'data/pv'

In [146]:
from ratelimit import limits, sleep_and_retry

# The API litmit is 50 requests per hour and 6 requests per minute that means 1 request per 72 seconds
@sleep_and_retry
@limits(calls=1, period=72)
def make_api_call(year: str, place: Place):
    # Construct the arguments for the API call
    args = {
            'lat': place.latitude,
            'lon': place.longitude,
            'date_from': str(year) + '-01-01',
            'date_to': str(year) + '-12-31',
            'dataset': 'merra2',
            'capacity': 1.0,
            'system_loss': 0.1,
            'tracking': 0,
            'tilt': 35,
            'azim': 180,
            'format': 'csv'
        }
    r = s.get(url, params=args)
    # Parse the CSV data from the response content
    csv_data = csv.reader(r.text.splitlines(), delimiter=',')
    # Write the CSV data to a file on disk
    try:
        with open(place.name[:2]+"/"+place.name+"_"+str(year)+'.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            # Skip first 3 lines in csv file because it contains the metadata
            for i in range(3):
                next(csv_data)
            for row in csv_data:
                writer.writerow(row)
        file.close()
    except:
        print("Error while writing the file "+place.name+"_"+str(year)+'.csv')

In [147]:
#collect the data from 1980 to 2020
for year in range(1980, 2021):
    #iterate over the locations
    for place in locations:
        # Check if file already exists
        if not os.path.exists(place.name[:2]+"/"+place.name+"_"+str(year)+'.csv'):
            print("Downloading data for "+place.name+" in "+str(year))
            make_api_call(year, place)

## Check if the downloaded data is complete
The downloaded data is checked if the file presents and the first column of the last row is 31.12 of the according year.

In [148]:
# Set the flag variable to indicate whether all files were downloaded correctly
all_downloaded = True
# List of not correctly downloaded files
not_downloaded = []

for year in range(1980, 2021):
    for place in locations:
        if not os.path.exists(place.name[:2] + "/" + place.name + "_" + str(year) + '.csv'):
            print("The file " + place.name + "_" + str(year) + '.csv' + " is not found")
            all_downloaded = False
        else:
            # Read the CSV file and extract the last row
            with open(place.name[:2] + "/" + place.name + "_" + str(year) + '.csv', mode='r') as file:
                reader = csv.reader(file,delimiter=',')
                rows = list(reader)
                if len(rows) == 0:
                    print("The file " + place.name + "_" + str(year) + '.csv' + " is empty")
                    all_downloaded = False
                    not_downloaded.append(place.name + "_" + str(year) + '.csv')
                else:
                    last_row = rows[-1]
                    # Check if the first column of the last row is year-12-31 23:00
                    if last_row[0] != str(year) + '-12-31 23:00':
                        print("The file " + place.name + "_" + str(year) + '.csv' + " is not downloaded correctly")
                        all_downloaded = False
                        not_downloaded.append(place.name + "_" + str(year) + '.csv')

# Print a message indicating whether all files were downloaded correctly
if all_downloaded:
    print("All files were downloaded correctly.")

The file GB0_1986.csv is empty
The file IE0_1987.csv is empty
The file IT3_1989.csv is empty
The file IT4_1989.csv is empty
The file LV0_1990.csv is empty
The file GB2_1996.csv is empty
The file FR5_1996.csv is empty
The file FR3_2001.csv is empty
The file DE2_2002.csv is empty
The file IT2_2005.csv is empty
The file SE0_2006.csv is empty
The file FR2_2010.csv is empty
The file DE3_2011.csv is empty
The file GB1_2015.csv is empty


## Redownload the data in case of incomplete data

In [149]:
# Redownload the files that were not downloaded correctly
for file in not_downloaded:
    print("Redownloading the file " + file)
    # Extract the name of the place and the year from the file name
    place_name = file.split("_")[0]
    year = file.split("_")[1].split(".")[0]
    # Find the place object with the name
    for place in locations:
        if place.name == place_name:
            make_api_call(year, place)

Redownloading the file GB0_1986.csv
Redownloading the file IE0_1987.csv
Redownloading the file IT3_1989.csv
Redownloading the file IT4_1989.csv
Redownloading the file LV0_1990.csv
Redownloading the file GB2_1996.csv
Redownloading the file FR5_1996.csv
Redownloading the file FR3_2001.csv
Redownloading the file DE2_2002.csv
Redownloading the file IT2_2005.csv
Redownloading the file SE0_2006.csv
Redownloading the file FR2_2010.csv
Redownloading the file DE3_2011.csv
Redownloading the file GB1_2015.csv
